## Dense retrieval using Haystack

In [ ]:
import pandas as pd


In [1]:
FAISS_INDEX_DIR = './faiss_indexes'

## Load pre-built index

In [2]:
EMBEDDING_MODELS = [("model1", "sentence-transformers/multi-qa-mpnet-base-dot-v1")]

embedding_model = EMBEDDING_MODELS[0]

In [3]:
# Load index
from haystack.document_stores import FAISSDocumentStore
index_path=f"{FAISS_INDEX_DIR}/{embedding_model[0]}/index.faiss"
config_path=f"{FAISS_INDEX_DIR}/{embedding_model[0]}/config.json"
document_store = FAISSDocumentStore.load(index_path=index_path, config_path=config_path)

# Check if the DocumentStore is loaded correctly
assert document_store.faiss_index_factory_str == "Flat"

In [4]:
document_store.get_document_count(), document_store.get_embedding_count()

(5364, 5364)

## Load retriever

In [6]:
from haystack.nodes import EmbeddingRetriever

retriever = EmbeddingRetriever(
    document_store=document_store,
    embedding_model=embedding_model[1],
)

/home/na/miniconda3/envs/vre/lib/python3.8/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


## Retrieve top k notebooks

In [13]:
k = 3
# Call the retrieve method to retrieve the top 10 documents for a given query
query = "congestion control"
retrieved_docs = retriever.retrieve(query=query, top_k=k*2)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
docids = []
scores = []
for doc in retrieved_docs: 
    docids.append(doc.meta['name'])
    scores.append(doc.score)

# Create a sample dataframe
df = pd.DataFrame({'docid': docids, 'score': scores})

# Group the scores by ID and apply max pooling
max_pooled_scores = df.groupby('docid')['score'].max()

# Sort the max pooled scores in descending order and select the top k records
k = 3  # Set the value of k to 3 (you can change it to any number you like)
top_k_scores = max_pooled_scores.sort_values(ascending=False).head(k)

# Create a list of dictionaries to show the top k scores with their corresponding document IDs
output_list = []
for docid, score in top_k_scores.items():
    output_dict = {'docid': docid, 'score': score}
    output_list.append(output_dict)

# Output the top k scores with their corresponding document IDs as a list of dictionaries
print(output_list)


In [20]:
top_k_df

,ID,Max Score
0,NB_eac81a61e9ec5f30d563da1267cce5bc77d7b5b5dee9140dddc3f901a2ff54f2,0.563438
1,NB_6e6405a806fb8dd4b7f3598cc1e8c50686b4ed08a6126cb2c323304c401a2390,0.563438
2,NB_1a630290ef2e92fc80032ca0bbfe166e2118ce3830083ee7742aee1a4a582cb6,0.561676
